<a href="https://colab.research.google.com/github/edyy-Z/Data-Analysis-Class-Works/blob/main/DataAnalysis_Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 4

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
df = pd.read_csv('sample_data/hospital_readmissions.csv')

In [ ]:
df.head()

,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
0,[70-80),8,72,1,18,2,0,0,Missing,Circulatory,Respiratory,Other,no,no,no,yes,no
1,[70-80),3,34,2,13,0,0,0,Other,Other,Other,Other,no,no,no,yes,no
2,[50-60),5,45,0,18,0,0,0,Missing,Circulatory,Circulatory,Circulatory,no,no,yes,yes,yes
3,[70-80),2,36,0,12,1,0,0,Missing,Circulatory,Other,Diabetes,no,no,yes,yes,yes
4,[60-70),1,42,0,7,0,0,0,InternalMedicine,Other,Circulatory,Respiratory,no,no,no,yes,no


**1-- Run a naive OLS regression on your time series data. Tell me how you expect your Xs to affect your Y and why.   Interpret your results.**



I am going to predict number of laboratory procedures performed during the hospital stay (n_lab_procedures)

In [ ]:
df[['n_lab_procedures']].describe()

,n_lab_procedures
count,25000.00000
mean,43.24076
std,19.81862
min,1.00000
25%,31.00000
50%,44.00000
75%,57.00000
max,113.00000


The next variable is days the patient stay in hospitals. It is measured on a scale ranging from 1 to 14.

In [ ]:
df[['time_in_hospital']].describe()

,time_in_hospital
count,25000.00000
mean,4.45332
std,3.00147
min,1.00000
25%,2.00000
50%,4.00000
75%,6.00000
max,14.00000


We will also look at whether the patient was readmitted at the hospital or not.  The readmitted indicator, which measures whether the patient was readmitted at the hospital ('yes or no'), can theoretically measure by 0 and 1.
0 indicates the patient was not readmitted at the hospital.
1 indicates the patient was readmitted at the hospital.


In [ ]:
df['readmitted'] = df['readmitted'].replace({'yes': 1, 'no': 0})
df[['readmitted']].describe()

<ipython-input-11-b746e70db9cc>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['readmitted'] = df['readmitted'].replace({'yes': 1, 'no': 0})


,readmitted
count,25000.000000
mean,0.470160
std,0.499119
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


I am now going to predict number of laboratory procedures perfomed during the hospital stay as a function of days stay and readmission. I expect patient with longer time stay and readmittion would have more laboratory procedures performed during the hospital stay.

In [ ]:
procedures = smf.ols(formula = 'n_lab_procedures ~ time_in_hospital + readmitted', data = df).fit()
print (procedures.summary())

                            OLS Regression Results                            
Dep. Variable:       n_lab_procedures   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     1516.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        21:48:27   Log-Likelihood:            -1.0871e+05
No. Observations:               25000   AIC:                         2.174e+05
Df Residuals:                   24997   BIC:                         2.174e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           33.2584      0.236  

Net of readmittion, a one-unit increase in time stay (time_in_hospital) is associated with a approximately 2.1626 increase in laboratory procedures.  Likewise, net of time stay, a 1 percentage point increase in readmission (radmitted) is associated with a approximately 0.7481 increase in lab procedures.

Overall, those two variables allow us to predict lab procedures with 10.8% more accuracy, compared to just guessing the relation.

**2-- Run a first differences regression on the same model in Question 1. Interpret your results. Do you draw a different conclusion than in Question 1? Explain.**

Now, let's look at the first differences of these relationships.  Will it continue to be the case?

In [ ]:
!pip install linearmodels

from linearmodels.panel import FirstDifferenceOLS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
columns = ['ccode', 'year', 'n_lab_procedures', 'time_in_hospital', 'readmitted']
df1 = df[columns].dropna()

KeyError: "['ccode', 'year'] not in index"

In [ ]:
# Set the MultiIndex for panel data
df1 = df1.set_index(['ccode', 'year'])

# Define the dependent and independent variables
y = df1['log_gle_cgdpc']
X = df1[['wbgi_pve', 'wdi_trade']]

# Fit the first-differenced panel data model
fdmodel = FirstDifferenceOLS(y, X)
results = fdmodel.fit(cov_type='clustered', cluster_entity=True)

print(results)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:          log_gle_cgdpc   R-squared:                        0.0063
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.0115
No. Observations:                1947   R-squared (Within):               0.0099
Date:                Sun, Jun 09 2024   R-squared (Overall):              0.0115
Time:                        17:37:24   Log-likelihood                    1537.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6.1914
Entities:                         174   P-value                           0.0021
Avg Obs:                       12.190   Distribution:                  F(2,1945)
Min Obs:                       1.0000                                           
Max Obs:                       13.000   F-statistic (robust):             3.6934
                            

Net of trade, a one-unit change in political stability (wbgi_pve) is associated with a 2.3% change in GDP from one period to the next.
Net of stability, a one percentage point change in trade (wdi_trade) is associated with a 0.06% change in GDP from one period to the next.  Note that the coefficients are smaller in magnitude compared to the levels model, which is expected since we're now looking at changes in GDP from one period to the next, rather than the overall levels of GDP.

Overall, we still see that political stability has a substantively significant and positive impact on GDP growth.  But trade has a very small positive and possibly only marginally significant impact on GDP growth.  Political stability appears much more related to GDP growth than trade.

Just so we can see how the first differences are distributed, take a look at them here.

In [ ]:
# Group by 'ccode' and calculate the first differences for the relevant columns

df1['log_gle_cgdpc_diff'] = df1.groupby('ccode')['log_gle_cgdpc'].diff()
df1['wbgi_pve_diff'] = df1.groupby('ccode')['wbgi_pve'].diff()
df1['wdi_trade_diff'] = df1.groupby('ccode')['wdi_trade'].diff()

In [ ]:
df1[['log_gle_cgdpc_diff', 'wbgi_pve_diff', 'wdi_trade_diff']].describe()

,log_gle_cgdpc_diff,wbgi_pve_diff,wdi_trade_diff
count,1947.000000,1947.000000,1947.000000
mean,0.050422,-0.008720,1.120139
std,0.098029,0.258724,11.209753
min,-0.903808,-1.846916,-79.133903
25%,0.009038,-0.140318,-2.980718
50%,0.045917,-0.000873,1.669658
75%,0.087945,0.128267,5.576761
max,0.931831,1.511317,132.216272
